In [1]:
from pathlib import Path
import re

import eelbrain
import mne
import trftools

In [2]:
STIMULI = [str(i) for i in range(1, 13)]
DATA_ROOT = Path("~").expanduser() / 'Dropbox (University of Michigan)' / 'Alice_trf' 
PREDICTOR_DIR = DATA_ROOT / 'predictors'
EEG_DIR = DATA_ROOT / 'eeg'
SUBJECTS = [path.name for path in EEG_DIR.iterdir() if re.match(r'S\d*', path.name)]
# Define a target directory for TRF estimates and make sure the directory is created
TRF_DIR = DATA_ROOT / 'TRFs'
TRF_DIR.mkdir(exist_ok=True)

In [3]:
DATA_ROOT

PosixPath('/Users/jobrenn/Dropbox (University of Michigan)/Alice_trf')

In [4]:
# Load stimuli
# ------------
# Make sure to name the stimuli so that the TRFs can later be distinguished
# Load the gammatone-spectrograms; use the time axis of these as reference
gammatone = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~gammatone-8.pickle') for stimulus in STIMULI]
# Resample the spectrograms to 100 Hz (time-step = 0.01 s), which we will use for TRFs
gammatone = [x.bin(0.01, dim='time', label='start') for x in gammatone]
# Pad onset with 100 ms and offset with 1 second; make sure to give the predictor a unique name as that will make it easier to identify the TRF later
gammatone = [trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='gammatone') for x in gammatone]
# Load the broad-band envelope and process it in the same way
envelope = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~gammatone-1.pickle') for stimulus in STIMULI]
envelope = [x.bin(0.01, dim='time', label='start') for x in envelope]
envelope = [trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='envelope') for x in envelope]
onset_envelope = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~gammatone-on-1.pickle') for stimulus in STIMULI]
onset_envelope = [x.bin(0.01, dim='time', label='start') for x in onset_envelope]
onset_envelope = [trftools.pad(x, tstart=-0.100, tstop=x.time.tstop + 1, name='onset') for x in onset_envelope]
# Load onset spectrograms and make sure the time dimension is equal to the gammatone spectrograms
gammatone_onsets = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~gammatone-on-8.pickle') for stimulus in STIMULI]
gammatone_onsets = [x.bin(0.01, dim='time', label='start') for x in gammatone_onsets]
gammatone_onsets = [eelbrain.set_time(x, gt.time, name='gammatone_on') for x, gt in zip(gammatone_onsets, gammatone)]
# Load word tables and convert tables into continuous time-series with matching time dimension
#word_tables = [eelbrain.load.unpickle(PREDICTOR_DIR / f'{stimulus}~word.pickle') for stimulus in STIMULI]
#word_onsets = [eelbrain.event_impulse_predictor(gt.time, ds=ds, name='word') for gt, ds in zip(gammatone, word_tables)]
# Function and content word impulses based on the boolean variables in the word-tables
#word_lexical = [eelbrain.event_impulse_predictor(gt.time, value='lexical', ds=ds, name='lexical') for gt, ds in zip(gammatone, word_tables)]
#word_nlexical = [eelbrain.event_impulse_predictor(gt.time, value='nlexical', ds=ds, name='non_lexical') for gt, ds in zip(gammatone, word_tables)]


In [5]:
# Extract the duration of the stimuli, so we can later match the EEG to the stimuli
durations = [gt.time.tmax for stimulus, gt in zip(STIMULI, gammatone)]

In [6]:
sum(durations)

735.24

In [7]:
# Models
# ------
# Pre-define models here to have easier access during estimation. In the future, additional models could be added here and the script re-run to generate additional TRFs.
models = {
    # Acoustic models
    'envelope': [envelope],
#    'envelope+onset': [envelope, onset_envelope],
#    'acoustic': [gammatone, gammatone_onsets],
    # Models with word-onsets and word-class
#    'words': [word_onsets],
#    'words+lexical': [word_onsets, word_lexical, word_nlexical],
#    'acoustic+words': [gammatone, gammatone_onsets, word_onsets],
#    'acoustic+words+lexical': [gammatone, gammatone_onsets, word_onsets, word_lexical, word_nlexical],
}

In [8]:
#Check raw eeg data

subject = 'S03'
raw = mne.io.read_raw(EEG_DIR / subject / f'{subject}_alice-raw.fif', preload=True)

In [9]:
raw.filter(0.5, 20)
raw.interpolate_bads()

<Raw | S03_alice-raw.fif, 63 x 367300 (734.6 s), ~176.6 MB, data loaded>

In [10]:
%matplotlib
raw.plot();

Using matplotlib backend: MacOSX


Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.9/site-packages/matplotlib/cbook/__init__.py", line 270, in process
    func(*args, **kwargs)
  File "/opt/anaconda3/lib/python3.9/site-packages/mne/viz/_mpl_figure.py", line 511, in _resize
    old_width, old_height = self.mne.fig_size_px
AttributeError: 'BrowserParams' object has no attribute 'fig_size_px'


In [13]:
# Estimate TRFs
# -------------
# Loop through subjects to estimate TRFs

run_subjects = ['S01', 'S03', 'S04']

for subject in run_subjects:
    subject_trf_dir = TRF_DIR / subject
    subject_trf_dir.mkdir(exist_ok=True)
    # Generate all TRF paths so we can check whether any new TRFs need to be estimated
    trf_paths = {model: subject_trf_dir / f'{subject} {model}.pickle' for model in models}

    # Skip this subject if all files already exist
    #if all(path.exists() for path in trf_paths.values()):
    #    continue
    # Load the EEG data
    raw = mne.io.read_raw(EEG_DIR / subject / f'{subject}_alice-raw.fif', preload=True)
    # Band-pass filter the raw data between 0.2 and 20 Hz
    raw.filter(0.5, 20)
    # Interpolate bad channels
    raw.interpolate_bads()
    # Extract the events marking the stimulus presentation from the EEG file
    events = eelbrain.load.fiff.events(raw)
    # Not all subjects have all trials; determine which stimuli are present
    trial_indexes = [STIMULI.index(stimulus) for stimulus in events['event']]

    # Extract the EEG data segments corresponding to the stimuli
    trial_durations = [durations[i] for i in trial_indexes]
    eeg = eelbrain.load.fiff.variable_length_epochs(events, -0.100, trial_durations, decim=5, connectivity='auto')

    # Since trials are of unequal length, we will concatenate them for the TRF estimation.
    eeg_concatenated = eelbrain.concatenate(eeg)

    for model, predictors in models.items():
        path = trf_paths[model]
        # Skip if this file already exists
        if path.exists():
            continue
        print(f"Estimating: {subject} ~ {model}")
        # Select and concetenate the predictors corresponding to the EEG trials
        predictors_concatenated = []
        for predictor in predictors:
            predictors_concatenated.append(eelbrain.concatenate([predictor[i] for i in trial_indexes]))
        # Fit the mTRF
        trf = eelbrain.boosting(eeg_concatenated, predictors_concatenated, -0.100, 1.000, error='l1', basis=0.050, partitions=5, test=1, selective_stopping=True)
        # Save the TRF for later analysis
        eelbrain.save.pickle(trf, path)

Estimating: S04 ~ envelope


Fitting models:   0%|          | 0/1220 [00:00<?, ?it/s]

### Sandbox

In [19]:
raw.info['sfreq']

500.0

In [20]:
events.info

{'raw': <Raw | S03_alice-raw.fif, 63 x 367300 (734.6 s), ~176.6 MB, data loaded>}

In [21]:
eeg_concatenated.get_dim('time')

UTS(0.0, 0.01, 73656)

In [22]:
predictors_concatenated[0].get_dim('time')

UTS(0.0, 0.01, 73656)